In [1]:
import py_stringsimjoin as ssj
from py_stringsimjoin.debugmatcher.debug_matcher import *
from py_stringsimjoin.labeler.labeler import *
from py_stringsimjoin.active_learner.active_learner import *
from sklearn.linear_model import LogisticRegression
import pandas as pd
import py_stringmatching as sm
from py_stringsimjoin.sampler.sample import *
from py_stringsimjoin.feature.autofeaturegen import *
from py_stringsimjoin.feature.extractfeatures import *

In [2]:
ws=sm.WhitespaceTokenizer(return_set=True)
qg3=sm.QgramTokenizer(qval=3,return_set=True)

In [3]:
ldf=pd.read_csv('http://pages.cs.wisc.edu/~anhai/data/corleone_data/citations/dblp.csv')
rdf=pd.read_csv('http://pages.cs.wisc.edu/~anhai/data/corleone_data/citations/google_scholar.csv')

In [4]:
import re
#ldf['cln_title'] = ldf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
#rdf['cln_title'] = rdf['title'].apply(lambda r: re.sub(r'[^\w\s]',' ',str(r).lower()))
l=[]
for idx, row in ldf.iterrows():
    s=re.sub(r'[^\w\s]',' ',str(row['title']).lower())
    if pd.notnull(row['authors']):
        s += re.sub(r'[^\w\s]',' ',str(row['authors']).lower())
    l.append(s)
r=[]
for idx, row in rdf.iterrows():
    s=re.sub(r'[^\w\s]',' ',str(row['title']).lower())
    if pd.notnull(row['authors']):
        s += re.sub(r'[^\w\s]',' ',str(row['authors']).lower())
    r.append(s)
ldf['cln_title'] = l
rdf['cln_title'] = r

In [5]:
seed=pd.read_csv('/scratch/citseed.csv')

In [6]:
c=sample_pairs(ldf,rdf,'id','id','cln_title','cln_title',100000,25,seed)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:02


In [7]:
len(c)

100004

In [8]:
labeled_c = label_table_using_gold(c, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_c['label'])

331

In [9]:
ft=get_features(['JACCARD'],{'ws':ws})
ft

,sim_measure_type,tokenizer,sim_function
feature_name,,,
jaccard_ws,JACCARD,<py_stringmatching.tokenizer.whitespace_tokenizer.WhitespaceTokenizer object at 0x7f5e147abc50>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...


In [10]:
#ft.ix['jaccard_ws']['tokenizer'] = qg3
fvs = extract_feature_vecs(c, 'l_id','r_id',ldf,rdf,'id','id','cln_title','cln_title',ft,n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:20


In [11]:
len(fvs)

100004

In [12]:
lr=LogisticRegression()
al=ActiveLearner(lr,20,20,'/scratch/citgold.csv',seed)

In [13]:
lp = al.learn(fvs, '_id', 'l_id','r_id')

                              l_id          r_id  jaccard_ws
_id                                                         
100000  conf/vldb/ChakrabartiDAR97  8IR1NrQ4zm8J    0.666667
100001  conf/vldb/ChakrabartiDAR97  DKUouEkK_b4J    0.000000
100002   conf/sigmod/LabrinidisR00  DKUouEkK_b4J    1.000000
100003   conf/sigmod/LabrinidisR00  8IR1NrQ4zm8J    0.000000
                              l_id          r_id  jaccard_ws  label
_id                                                                
100000  conf/vldb/ChakrabartiDAR97  8IR1NrQ4zm8J    0.666667      1
100001  conf/vldb/ChakrabartiDAR97  DKUouEkK_b4J    0.000000      0
100002   conf/sigmod/LabrinidisR00  DKUouEkK_b4J    1.000000      1
100003   conf/sigmod/LabrinidisR00  8IR1NrQ4zm8J    0.000000      0
Selecting next batch...
computing entropy
sorting
sampling
[0.69314718010930676, 0.69314718010930676, 0.69314718010930676, 0.69314718010930676, 0.6931471148005226, 0.6931471148005226, 0.6931471148005226, 0.6931471148005226, 0.

py_stringsimjoin/active_learner/active_learner.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  to_be_labeled_pairs['label'] = labels


In [14]:
c.head()

,_id,l_id,r_id,label
0,0,conf/vldb/Baumann03,eq-98HcgqTgJ,0
1,1,journals/sigmod/GrayG97,eq-98HcgqTgJ,0
2,2,conf/sigmod/CareyDNAGS97,eq-98HcgqTgJ,0
3,3,conf/vldb/OzdenBRS94,eq-98HcgqTgJ,0
4,4,conf/vldb/BrownS95,eq-98HcgqTgJ,0


In [15]:
lp.reset_index(level=0, inplace=True)
sum(lp['label'])

169

In [16]:
ft1=get_features(['JACCARD'])
ft1

,sim_measure_type,tokenizer,sim_function
feature_name,,,
jaccard_qg3,JACCARD,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f5e14280790>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...
jaccard_ws,JACCARD,<py_stringmatching.tokenizer.whitespace_tokenizer.WhitespaceTokenizer object at 0x7f5e14280450>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...
jaccard_qg2,JACCARD,<py_stringmatching.tokenizer.qgram_tokenizer.QgramTokenizer object at 0x7f5e14280250>,<bound method Jaccard.get_raw_score of <py_stringmatching.similarity_measure.jaccard.Jaccard obj...


In [17]:
reload(ssj.feature.extractfeatures)
lp1=lp.copy()
del lp1['jaccard_ws']
fvs1 = extract_feature_vecs(lp1, 'l_id','r_id',ldf,rdf,'id','id','cln_title','cln_title',ft1,n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [18]:
len(set(fvs1.index.values))

404

In [19]:
fvs1['label'] = lp1['label']

In [22]:
from py_stringsimjoin.matcher.select_matcher import select_matcher
reload(ssj.matcher.select_matcher)
out=select_matcher(fvs1, 'label', ft1, metric='f1')
out['cv_stats']

,Name,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,jaccard_qg2,5,0.84375,0.800000,0.861538,0.828571,0.828571,0.832486
1,jaccard_qg3,5,0.87500,0.851852,0.865672,0.885714,0.805970,0.856842
2,jaccard_ws,5,0.83871,0.754717,0.769231,0.811594,0.709677,0.776786


In [147]:
from py_stringsimjoin.matcher.matcher_utils import split_train_test
splits = split_train_test(fvs1, 'label')
P =splits['train']
Q=splits['test']

In [107]:
type(out['selected_matcher'])

sklearn.linear_model.logistic.LogisticRegression

In [109]:
lr1=LogisticRegression()
lr1.intercept_
#dbg = debug_matcher(lr1, P, Q, 'jaccard_qg3', 'label', 'l_id', 'r_id', ldf, rdf, 'id', 'id')

AttributeError: 'LogisticRegression' object has no attribute 'intercept_'

In [51]:
dbg['false_neg'][['l_title', 'r_title', 'jaccard_qg3']]

,l_title,r_title,jaccard_qg3
61,The New Middleware,The new standard,0.153846
61,Query Flocks: A Generalization of Association-Rule Mining,"A Generalization of Association-Rule Mining, D",0.311111
33,Communication Efficient Distributed Mining of Association Rules,A high-performance distributed algorithm for mining association rules,0.412698
33,Lineage tracing for general data warehouse transformations,Tracing the lineage of vlew data in a warehousing environment,0.327586
72,Versioning and Configuration Management in an Object-Oriented Data Model,An Active and Deductive Object-Oriented Data Model,0.283019
72,Mining Quantitative Association Rules in Large Relational Tables,Mining association rules between sets ofitems in large databases,0.328358
81,Sybase Replication Server,Sybase replication Server,0.435484
81,Checks and Balances: Monitoring Data Quality Problems in Network Traffic Databases,IPSOFACTO: A Visual Correlation Tool for Aggregate Network Traffic Data,0.300613
74,XBase: making your gigabyte disk queriable,XBase. Making Your Gigabyte Disk Files Queriable [Z],0.425926
74,Engineering Federated Information Systems: Report of EFIS '99 Workshop,Engineering Federated Information Systems: Report of EFIS'99 Workshop.,0.436709


In [52]:
dbg['stats']

{'f1': 0.5554106910039114, 'precision': 0.5803814713896458, 'recall': 0.5325}

In [22]:
-lr1.intercept_/lr1.coef_

array([[ 0.61488554]])

In [42]:
len(dbg['false_neg']), len(dbg['false_pos'])

(19, 7)

In [25]:
lr2=LogisticRegression()
lr2.fit(fvs1[['jaccard_qg3']].values, fvs1['label'].values)
-lr2.intercept_/lr2.coef_

array([[ 0.55374328]])

In [59]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
lsvc.fit(lp[['jaccard_ws']].values, lp['label'].values)
-lsvc.intercept_/lsvc.coef_

array([[ 0.56236397]])

In [65]:
what_if_matrix(lp, 'jaccard_ws', 'label', 0.57, 0.02, 10)

,threshold,precision,recall,f1
0,0.37,0.557377,0.988372,0.712788
1,0.39,0.599291,0.982558,0.744493
2,0.41,0.639847,0.970930,0.771363
3,0.43,0.670683,0.970930,0.793349
4,0.45,0.694561,0.965116,0.807786
5,0.47,0.723684,0.959302,0.825000
6,0.49,0.752336,0.936047,0.834197
7,0.51,0.791878,0.906977,0.845528
8,0.53,0.812834,0.883721,0.846797
9,0.55,0.808989,0.837209,0.822857


In [29]:
lp1=lp[lp['jaccard_ws'] >= 0.34]
len(lp1)

376

In [30]:
lp1.head()

,_id,l_id,r_id,jaccard_ws,label
0,100000,conf/vldb/ChakrabartiDAR97,8IR1NrQ4zm8J,0.784000,1
2,100002,conf/sigmod/LabrinidisR00,DKUouEkK_b4J,1.000000,1
24,37211,journals/sigmod/EnglertGH95,JRXuKm3E_foJ,1.000000,1
25,5067,journals/sigmod/NobleS95,rRvcD6HKhasJ,0.941176,1
26,61027,journals/tods/Chomicki03,pHuL-9kHWH0J,1.000000,1


In [51]:
cc=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',qg3,0.5777797,l_out_attrs=['year'], r_out_attrs=['year'],n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:39


In [53]:
len(cc)

5070

In [54]:
labeled_cc = label_table_using_gold(cc, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_cc['label'])


4626

In [55]:
4626/5070.0, 4626/5347.0

(0.9124260355029585, 0.8651580325416122)

In [56]:
cc1=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',qg3,0.34,l_out_attrs=['year'], r_out_attrs=['year'],n_jobs=4)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:02:00


In [57]:
labeled_cc1 = label_table_using_gold(cc1, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_cc1['label'])

5270

In [26]:
cc2=ssj.jaccard_join(ldf,rdf,'id','id','cln_title','cln_title',qg3,0.55374328,n_jobs=4)


0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:50


In [27]:
labeled_cc2 = label_table_using_gold(cc2, 'l_id', 'r_id', '/scratch/citgold.csv')
sum(labeled_cc2['label'])

4752

In [28]:
len(cc2)

5340

In [29]:
4752/5340.0, 4752/5347.0

(0.8898876404494382, 0.8887226482139517)